In [587]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import platform
import seaborn as sns
import os

In [588]:
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

print(platform.system()) # 플랫폼 확인

if platform.system() == 'Windows': # Window
    matplotlib.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    matplotlib.rc('font', family='AppleGothic')

Darwin


In [589]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [590]:
folder = os.listdir()

file = []
for f in folder:
    if "jinju" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
# gajwa = gajwa.dropna(axis=0)

In [591]:
gajwa.도로조건.value_counts()

8m미만     1797
-        1581
12m미만    1454
25m미만     384
25m이상      55
Name: 도로조건, dtype: int64

In [592]:
geo = pd.read_csv("./geocoding.csv")
gajwa["경도"] = geo.X
gajwa["위도"] = geo.Y

gajwa = gajwa.fillna({'건축년도':gajwa['건축년도'].mode()[0]})
gajwa = gajwa.fillna({'도로명':gajwa['도로명'].mode()[0]})
gajwa = gajwa.fillna({'도로조건':gajwa['도로조건'].mode()[0]})

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
         a = "8"
    road.append(int(a.split("m")[0]))
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day

# 위치 column만들기
gajwa["location"] = gajwa.시군구 + " " + gajwa.도로명

# 계약년월 기준으로 전월세전환율 입력하기
rate = pd.read_csv("./rate.csv", encoding="cp949")
rate = rate.T
rate = rate[3:]

month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep


# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["전세전환가격"] = value
#gajwa["도로별 전세값평균"] = gajwa.groupby("도로명").전세전환가격.transform("mean")


# 도로명 결측치 제거
indexNames = []
for i in range(len(gajwa["도로명"])):
    if "번길" not in gajwa["도로명"][i]:
        indexNames.append(i)

gajwa.drop(indexNames, inplace=True)


gajwa

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세전환율,전세전환가격
0,경상남도 진주시 가좌동,1***,25,12.00,월세,201902,16,200,35,2016,가좌길74번길,128.105889,35.157466,2019,2,경상남도 진주시 가좌동 가좌길74번길,5.1,8435
1,경상남도 진주시 가좌동,1***,12,14.85,월세,201902,20,500,35,2019,가좌길64번길,128.106121,35.158214,2019,2,경상남도 진주시 가좌동 가좌길64번길,5.1,8735
3,경상남도 진주시 가좌동,1***,8,16.00,전세,201902,20,3000,0,2016,가좌길48번길,128.106188,35.158872,2019,2,경상남도 진주시 가좌동 가좌길48번길,5.1,3000
4,경상남도 진주시 가좌동,1***,8,18.00,전세,201902,20,5000,0,2016,가좌길44번길,128.106188,35.158872,2019,2,경상남도 진주시 가좌동 가좌길44번길,5.1,5000
5,경상남도 진주시 가좌동,1***,8,18.00,전세,201902,21,5000,0,2019,진주대로404번길,128.106188,35.158872,2019,2,경상남도 진주시 가좌동 진주대로404번길,5.1,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,경상남도 진주시 호탄동,6**,12,33.00,월세,202011,19,100,23,2002,호탄길21번길,128.104882,35.150661,2020,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972
5253,경상남도 진주시 호탄동,6**,8,42.90,전세,202011,25,7500,0,2018,호탄길9번길,128.115149,35.162316,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,7500
5254,경상남도 진주시 호탄동,9**,12,50.00,월세,202011,10,5000,30,2011,가호로61번길,128.103328,35.145290,2020,11,경상남도 진주시 호탄동 가호로61번길,4.7,12660
5255,경상남도 진주시 호탄동,6**,8,50.00,전세,202011,18,9000,0,2018,호탄길9번길,128.105966,35.156961,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,9000


In [593]:
corr = gajwa.corr()
corr

,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,경도,위도,year,month,전세전환가격
도로조건,1.000000,-0.008062,0.190131,-0.007537,-0.037119,0.096157,0.007129,0.046169,0.024739,0.189510,0.039660,0.171422
계약면적(㎡),-0.008062,1.000000,-0.149413,-0.014312,0.449450,0.056091,-0.075128,0.251820,0.124425,-0.150664,0.098479,0.439190
계약년월,0.190131,-0.149413,1.000000,-0.019595,-0.030070,-0.005450,0.175812,0.034178,0.013560,0.999910,-0.028032,0.277357
계약일,-0.007537,-0.014312,-0.019595,1.000000,0.002483,-0.016596,0.012419,-0.003748,-0.017213,-0.020223,0.047498,-0.020224
보증금(만원),-0.037119,0.449450,-0.030070,0.002483,1.000000,-0.688701,0.082603,0.163282,0.096386,-0.031438,0.103046,0.016563
월세(만원),0.096157,0.056091,-0.005450,-0.016596,-0.688701,1.000000,0.005268,0.003626,-0.012725,-0.004681,-0.057145,0.651482
건축년도,0.007129,-0.075128,0.175812,0.012419,0.082603,0.005268,1.000000,-0.012462,-0.009713,0.175849,-0.008837,0.122126
경도,0.046169,0.251820,0.034178,-0.003748,0.163282,0.003626,-0.012462,1.000000,0.682936,0.033617,0.040693,0.154574
위도,0.024739,0.124425,0.013560,-0.017213,0.096386,-0.012725,-0.009713,0.682936,1.000000,0.013209,0.025696,0.070627
year,0.189510,-0.150664,0.999910,-0.020223,-0.031438,-0.004681,0.175849,0.033617,0.013209,1.000000,-0.041432,0.276658


In [594]:
# # 전세 월세 구분 데이터셋
# gajwa = gajwa[gajwa.전월세구분 == "월세"]
# gajwa = pd.DataFrame.drop(gajwa, columns=["전월세구분"], axis=1)

#원핫인코딩
gajwa = pd.get_dummies(gajwa, columns = ["도로조건"])
gajwa = pd.get_dummies(gajwa, columns = ["전월세구분"])

gajwa

,시군구,번지,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,...,year,month,location,전월세전환율,전세전환가격,도로조건_8,도로조건_12,도로조건_25,전월세구분_월세,전월세구분_전세
0,경상남도 진주시 가좌동,1***,12.00,201902,16,200,35,2016,가좌길74번길,128.105889,...,2019,2,경상남도 진주시 가좌동 가좌길74번길,5.1,8435,0,0,1,1,0
1,경상남도 진주시 가좌동,1***,14.85,201902,20,500,35,2019,가좌길64번길,128.106121,...,2019,2,경상남도 진주시 가좌동 가좌길64번길,5.1,8735,0,1,0,1,0
3,경상남도 진주시 가좌동,1***,16.00,201902,20,3000,0,2016,가좌길48번길,128.106188,...,2019,2,경상남도 진주시 가좌동 가좌길48번길,5.1,3000,1,0,0,0,1
4,경상남도 진주시 가좌동,1***,18.00,201902,20,5000,0,2016,가좌길44번길,128.106188,...,2019,2,경상남도 진주시 가좌동 가좌길44번길,5.1,5000,1,0,0,0,1
5,경상남도 진주시 가좌동,1***,18.00,201902,21,5000,0,2019,진주대로404번길,128.106188,...,2019,2,경상남도 진주시 가좌동 진주대로404번길,5.1,5000,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,경상남도 진주시 호탄동,6**,33.00,202011,19,100,23,2002,호탄길21번길,128.104882,...,2020,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972,0,1,0,1,0
5253,경상남도 진주시 호탄동,6**,42.90,202011,25,7500,0,2018,호탄길9번길,128.115149,...,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,7500,1,0,0,0,1
5254,경상남도 진주시 호탄동,9**,50.00,202011,10,5000,30,2011,가호로61번길,128.103328,...,2020,11,경상남도 진주시 호탄동 가호로61번길,4.7,12660,0,1,0,1,0
5255,경상남도 진주시 호탄동,6**,50.00,202011,18,9000,0,2018,호탄길9번길,128.105966,...,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,9000,1,0,0,0,1


## 지도

In [595]:
import math as m

def distance(x1, x2, y1, y2):
    radius = 6371 # 지구 반지름(km)
    toRadian = m.pi / 180

    deltaLatitude = abs(x1 - x2) * toRadian
    deltaLongitude = abs(y1 - y2) * toRadian

    sinDeltaLat = m.sin(deltaLatitude / 2)
    sinDeltaLng = m.sin(deltaLongitude / 2)
    squareRoot = m.sqrt(sinDeltaLat * sinDeltaLat + m.cos(x1 * toRadian) * m.cos(x2 * toRadian) * sinDeltaLng * sinDeltaLng)

    distance = 2 * radius * m.asin(squareRoot)

    return distance

In [596]:
gnu_lat = 35.1530444
gnu_long = 128.1010899

gnu_dist = []
for long, lat in zip(gajwa["경도"], gajwa["위도"]):
    d = distance(gnu_lat, lat, gnu_long, long)
    gnu_dist.append(d)
gajwa["gnu_distance"] = gnu_dist

In [597]:
def remove_out(dataframe, remove_col):
    dff = dataframe
    for k in remove_col:
        level_1q = dff[k].quantile(0.25)
        level_3q = dff[k].quantile(0.7)
        IQR = level_3q - level_1q
        rev_range = 3  # 제거 범위 조절 변수
        dff = dff[(dff[k] <= level_3q + (rev_range * IQR)) & (dff[k] >= level_1q - (rev_range * IQR))]
        dff = dff.reset_index(drop=True)
    return dff

gajwa = remove_out(gajwa, ["전세전환가격", "gnu_distance"])

gajwa

,시군구,번지,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,...,month,location,전월세전환율,전세전환가격,도로조건_8,도로조건_12,도로조건_25,전월세구분_월세,전월세구분_전세,gnu_distance
0,경상남도 진주시 가좌동,1***,12.00,201902,16,200,35,2016,가좌길74번길,128.105889,...,2,경상남도 진주시 가좌동 가좌길74번길,5.1,8435,0,0,1,1,0,0.657356
1,경상남도 진주시 가좌동,1***,14.85,201902,20,500,35,2019,가좌길64번길,128.106121,...,2,경상남도 진주시 가좌동 가좌길64번길,5.1,8735,0,1,0,1,0,0.734594
2,경상남도 진주시 가좌동,1***,16.00,201902,20,3000,0,2016,가좌길48번길,128.106188,...,2,경상남도 진주시 가좌동 가좌길48번길,5.1,3000,1,0,0,0,1,0.796668
3,경상남도 진주시 가좌동,1***,18.00,201902,20,5000,0,2016,가좌길44번길,128.106188,...,2,경상남도 진주시 가좌동 가좌길44번길,5.1,5000,1,0,0,0,1,0.796668
4,경상남도 진주시 가좌동,1***,18.00,201902,21,5000,0,2019,진주대로404번길,128.106188,...,2,경상남도 진주시 가좌동 진주대로404번길,5.1,5000,1,0,0,0,1,0.796668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3837,경상남도 진주시 호탄동,6**,33.00,202011,16,1000,40,2016,호탄길34번길,128.102026,...,11,경상남도 진주시 호탄동 호탄길34번길,4.7,11213,1,0,0,1,0,0.440456
3838,경상남도 진주시 호탄동,6**,33.00,202011,19,100,23,2002,호탄길21번길,128.104882,...,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972,0,1,0,1,0,0.434847
3839,경상남도 진주시 호탄동,9**,50.00,202011,10,5000,30,2011,가호로61번길,128.103328,...,11,경상남도 진주시 호탄동 가호로61번길,4.7,12660,0,1,0,1,0,0.885957
3840,경상남도 진주시 호탄동,6**,50.00,202011,18,9000,0,2018,호탄길9번길,128.105966,...,11,경상남도 진주시 호탄동 호탄길9번길,4.7,9000,1,0,0,0,1,0.621416


In [598]:
# import folium

# m = folium.Map(location=[gnu_lat, gnu_long], zoom_start=16)

# for lo, la, in zip(gajwa.경도, gajwa.위도):
#     folium.Marker([la, lo]).add_to(m)

# m

In [599]:
# m = folium.Map(location=[gajwa.위도.max(),gajwa.경도.min()], zoom_start=16)
# folium.Marker([gajwa.위도.max(),gajwa.경도.min()]).add_to(m)
# m

In [600]:
gajwa_data = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "번지", "도로명", "계약일", "보증금(만원)", "월세(만원)", "location", "경도", "위도", "전월세전환율", "gnu_distance"], axis=1)

#gajwa_data.distance = gajwa_data.distance.transform(lambda x: (x - x.mean()) / x.std())
#gajwa_data.전세전환가격 = gajwa_data.전세전환가격.transform(lambda x: (x - x.mean()) / x.std())
gajwa_data

,계약면적(㎡),건축년도,year,month,전세전환가격,도로조건_8,도로조건_12,도로조건_25,전월세구분_월세,전월세구분_전세
0,12.00,2016,2019,2,8435,0,0,1,1,0
1,14.85,2019,2019,2,8735,0,1,0,1,0
2,16.00,2016,2019,2,3000,1,0,0,0,1
3,18.00,2016,2019,2,5000,1,0,0,0,1
4,18.00,2019,2019,2,5000,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
3837,33.00,2016,2020,11,11213,1,0,0,1,0
3838,33.00,2002,2020,11,5972,0,1,0,1,0
3839,50.00,2011,2020,11,12660,0,1,0,1,0
3840,50.00,2018,2020,11,9000,1,0,0,0,1


# 머신러닝

In [601]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
import catboost
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [602]:
X = gajwa_data.drop(["전세전환가격"], axis=1)
y = gajwa_data['전세전환가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

# minmaxScaler = MinMaxScaler()
# X_train_standardized2 = minmaxScaler.fit_transform(X_train)
# X_test_standardized2 = minmaxScaler.fit_transform(X_test)

## 선형회귀분석

In [603]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)
# y_preds = lr.predict(X_test)

# mse = mean_squared_error(y_test, y_preds)
# rmse = np.sqrt(mse)

# print(f"MSE : {mse:.3f}", f"RMSE: {rmse:.3f}")
# print(f"variance score : {r2_score(y_test, y_preds):.3f}")

In [604]:
# lr.fit(X_train_standardized, y_train)
# lr.score(X_test_standardized, y_test)

## Ridge

In [605]:
# R_model = Ridge()
# R_model.fit(X_train, y_train)
# print(R_model.score(X_test, y_test))

# R_model.fit(X_train_standardized, y_train)
# R_model.score(X_test_standardized, y_test)

## Lasso

In [606]:
# L_model = Lasso(alpha=0.5)
# L_model.fit(X_train, y_train)
# print(L_model.score(X_test, y_test))

# L_model.fit(X_train_standardized, y_train)
# L_model.score(X_test_standardized, y_test)

## ElasticNet

In [607]:
# ela = ElasticNet(alpha=0.5)
# ela.fit(X_train, y_train)
# print(ela.score(X_test, y_test))

# ela.fit(X_train_standardized, y_train)
# ela.score(X_test_standardized, y_test)

## 랜덤포레스트

In [608]:
randomforest = RandomForestRegressor(random_state=0, n_jobs=-1)
randomforest_model = randomforest.fit(X_train, y_train)
print(randomforest_model.score(X_test, y_test))

randomforest_model = randomforest.fit(X_train_standardized, y_train)
print(randomforest_model.score(X_test_standardized, y_test))

0.7097745747632915
0.6898541234497684


In [609]:
gbm = GradientBoostingRegressor()

gbm.fit(X_train, y_train)
print(gbm.score(X_test, y_test))

gbm.fit(X_train_standardized, y_train)
print(gbm.score(X_test_standardized, y_test))

0.706291187572216
0.6964570878250333


## LightGBM

In [610]:
lightgbm = lgb.LGBMRegressor()

lightgbm.fit(X_train, y_train)
print(lightgbm.score(X_test, y_test))

lightgbm.fit(X_train_standardized, y_train)
print(lightgbm.score(X_test_standardized, y_test))

0.7306618111720664
0.7098308409582601


## XGBoost

In [611]:
xgboost = XGBRegressor(n_estimators=50)
xgboost.fit(X_train, y_train)
print(xgboost.score(X_test, y_test))

xgboost.fit(X_train_standardized, y_train)
xgboost.score(X_test_standardized, y_test)

0.6994435102884722


/Users/chance/opt/anaconda3/envs/py3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.6826174301444816

## CatBoost

In [612]:
cb = catboost.CatBoostRegressor()
cb.fit(X_train_standardized, y_train)
cb.score(X_test_standardized, y_test)

Learning rate set to 0.048889
0:	learn: 2629.1389499	total: 1.01ms	remaining: 1s
1:	learn: 2559.0508662	total: 2.11ms	remaining: 1.05s
2:	learn: 2495.9308279	total: 5.09ms	remaining: 1.69s
3:	learn: 2440.1069691	total: 6.76ms	remaining: 1.68s
4:	learn: 2387.5602673	total: 9.4ms	remaining: 1.87s
5:	learn: 2330.0171464	total: 10.2ms	remaining: 1.69s
6:	learn: 2278.7054270	total: 11.1ms	remaining: 1.58s
7:	learn: 2229.6997149	total: 11.8ms	remaining: 1.47s
8:	learn: 2186.3094232	total: 12.5ms	remaining: 1.38s
9:	learn: 2146.0492625	total: 13.5ms	remaining: 1.33s
10:	learn: 2108.9378603	total: 14.3ms	remaining: 1.28s
11:	learn: 2072.5352508	total: 14.8ms	remaining: 1.22s
12:	learn: 2038.9160751	total: 15.5ms	remaining: 1.18s
13:	learn: 2010.1583521	total: 16.5ms	remaining: 1.16s
14:	learn: 1979.9735175	total: 17.4ms	remaining: 1.14s
15:	learn: 1953.1884420	total: 18.2ms	remaining: 1.12s
16:	learn: 1927.7308399	total: 18.8ms	remaining: 1.09s
17:	learn: 1903.7034504	total: 19.8ms	remaining: 

0.7082477744664037

In [613]:
gajwa_data.corr()

,계약면적(㎡),건축년도,year,month,전세전환가격,도로조건_8,도로조건_12,도로조건_25,전월세구분_월세,전월세구분_전세
계약면적(㎡),1.000000,-0.081107,-0.110038,0.088510,0.381677,0.057607,-0.063941,0.011087,-0.018334,0.018334
건축년도,-0.081107,1.000000,0.187470,0.003035,0.150912,-0.059980,0.077382,-0.038110,-0.020400,0.020400
year,-0.110038,0.187470,1.000000,-0.040994,0.327037,-0.252721,0.225013,0.087796,-0.024744,0.024744
month,0.088510,0.003035,-0.040994,1.000000,0.019830,-0.042898,0.034097,0.024977,-0.044145,0.044145
전세전환가격,0.381677,0.150912,0.327037,0.019830,1.000000,-0.195605,0.169855,0.078537,0.481488,-0.481488
도로조건_8,0.057607,-0.059980,-0.252721,-0.042898,-0.195605,1.000000,-0.920296,-0.273814,-0.048879,0.048879
도로조건_12,-0.063941,0.077382,0.225013,0.034097,0.169855,-0.920296,1.000000,-0.124281,0.029068,-0.029068
도로조건_25,0.011087,-0.038110,0.087796,0.024977,0.078537,-0.273814,-0.124281,1.000000,0.052512,-0.052512
전월세구분_월세,-0.018334,-0.020400,-0.024744,-0.044145,0.481488,-0.048879,0.029068,0.052512,1.000000,-1.000000
전월세구분_전세,0.018334,0.020400,0.024744,0.044145,-0.481488,0.048879,-0.029068,-0.052512,-1.000000,1.000000


## HyperParameter